In [4]:
import pandas as pd

In [1]:
fasta = {}
with open("AT_FC_contigs_discarded.fa") as file_one:
    for line in file_one:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            active_sequence_name = line[1:]
            if active_sequence_name not in fasta:
                fasta[active_sequence_name] = []
            continue
        sequence = line
        fasta[active_sequence_name].append(sequence)



In [2]:
#print(fasta)
dict_keys = fasta.items()
sorted_items = sorted(dict_keys)
len(sorted_items) # fasta files
len(sorted_items[0][1][0])

3360

In [5]:
pd.options.display.float_format = '{:,.0f}'.format
data = pd.read_csv("pos_data.csv", header = 0, index_col = 0)
df_list = data.values.tolist()
k = [list(filter(lambda x : str(x) !='nan',i)) for i in df_list]
len(k)


411

In [6]:
alt_data = pd.read_csv("alt_data.csv", header = 0, index_col = 0)
#alt_data
alt_list = alt_data.values.tolist()
m = [list(filter(lambda x : str(x) !='nan',i)) for i in alt_list]
len(m)

411

In [7]:
from Bio.Seq import Seq
import pandas as pd


def mutate_seq(seq, new_pos):
    mut_seq = seq.tomutable()
    for i in range(0, len(new_pos)):
        #print(alt[i])
        #print(pos[i])
        mut_seq[new_pos[i]] = alt[i]
    mut_seq = mut_seq.toseq()
    #print(mut_seq)
    return mut_seq

def find_diff_pro(pro, mut_pro):
    idx = []
    nuc_start = []
    nuc_end = []
    str_pro = []
    str_mut_pro = []
    
    for i in range(0,len(pro)):
        if pro[i] != mut_pro[i]:
            idx.append(i+1)
            str_pro.append(pro[i])
            str_mut_pro.append(mut_pro[i])
    
    for i in range(0, len(idx)):
        nuc_start.append((idx[i]*3)-2)
        nuc_end.append(idx[i]*3)
    
    d = {"pro_idx" :idx, "nuc_start": nuc_start, "nuc_end": nuc_end, "pro_seq" : str_pro, "mut_pro_seq" : str_mut_pro}
    df = pd.DataFrame(data = d)
    
    return df

def nonsyn_info(fasta, alt, pos):
    seq = Seq(fasta)
    pro_seq = seq.translate()
    new_pos = []
    for i in range(0, len(pos)):
        new_pos.append(pos[i]-1)
    mut_seq = mutate_seq(seq, new_pos)
    mut_pro_seq = mut_seq.translate()
    df = find_diff_pro(pro_seq, mut_pro_seq)
    return df
    
def find_nonsyn_snp_pos(df, pos):
    vals = []
    for i in range(0,len(df)):
        #print(i)
        for j in range(0, len(pos)):
            if int(pos[j]) in range(int(df["nuc_start"].iloc[i]), int(df["nuc_end"].iloc[i])+1):
                vals.append(pos[j])
        else: continue
    return vals

In [8]:
all_nonsyn_pos = []

for i in range(len(k)):
    fasta = sorted_items[i][1][0]
    pos = k[i]
    pos2 = [int(x) for x in pos]
    alt = m[i]
    ex = nonsyn_info(fasta, alt, pos2)
    all_nonsyn_pos.append(find_nonsyn_snp_pos(ex, pos2))


C:\Users\Aysevil\Anaconda3\lib\site-packages\Bio\Seq.py:2859: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [11]:
len(all_nonsyn_pos[1])

3

In [17]:
df = pd.DataFrame(all_nonsyn_pos)
df = df.transpose()
df

,0,1,2,3,4,5,6,7,8,9,...,401,402,403,404,405,406,407,408,409,410
0,43,127,90,207,40,85,224,73,17,26,...,273,11,181,70,56,41,29,54,20,107
1,55,707,271,386,42,116,262,94,38,60,...,286,13,183,220,136,54,92,55,55,226
2,67,934,541,430,45,247,290,107,39,112,...,334,28,202,239,227,62,184,56,56,266
3,136,nan,623,985,49,286,323,108,53,121,...,340,65,248,257,228,74,189,71,90,313
4,169,nan,624,"1,083",51,494,352,119,73,212,...,343,100,260,258,253,77,192,103,107,325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
300,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
301,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
302,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [18]:
df.to_csv("all_contigs_nonsynINFO.csv", index = False)

In [ ]:
###fixed allele